In [1]:
import pandas as pd
import numpy as np
import networkx as nx

collection_df = pd.read_csv("data/Pulsquares_transactions.csv")

# take log of transaction values to make easier to parse
values = collection_df['transaction_value']
result = np.where(values > 0.0000000001, values, 0)
collection_df['transaction_value'] = np.log(result, out=result, where=result > 0)

# reduce node ids
nodes_1 = collection_df['from_address'].to_numpy()
nodes_2 = collection_df['to_address'].to_numpy()
nodes = np.append(nodes_1, nodes_2)
nodes = set(nodes)
nodes_map = {node:indx for indx, node in enumerate(nodes)}

new_node_ids = collection_df[['from_address', 'to_address']].applymap(lambda x: nodes_map[x])
collection_df[['from_address', 'to_address']] = new_node_ids

# create column to flag riskiness
collection_array = collection_df.to_numpy()
collection_array_edited = np.delete(collection_array, 0, 1) # delete that weird unnamed column
flag_col = np.ones(len(collection_array_edited))
collection_array_edited = np.c_[collection_array_edited, flag_col]
collection_array_edited[:,9] = 'blue'

# generate graph for each token/collection pairing
token_graphs = {}
for i in collection_array_edited:
    if (i[3], i[4]) not in token_graphs: #<-- this is where we would add logic to include the address, and token id
        G = nx.MultiDiGraph()
        G.add_weighted_edges_from([(i[5], i[6], i[7])])
        token_graphs[(i[3], i[4])] = {'graph': G}
    else:
        token_graphs[(i[3], i[4])]['graph'].add_weighted_edges_from([(i[5], i[6], i[7])])

# ID wash trades
for i in token_graphs:
    all_components = nx.strongly_connected_components(token_graphs[i]['graph'])
    token_graphs[i]['strong components'] = [i for i in list(all_components) if len(i) == 2]

    # tally risky transactions for each token
    if token_graphs[i]['strong components'] != []: # if no strong components
        for edge in token_graphs[i]['strong components']:
            src, targ = tuple(edge)
            address = i[0] 
            token = i[1]
            inds = np.where((collection_array_edited[:,3] == [i[0]]) &\
                (collection_array_edited[:,4] == i[1]) &\
                (collection_array_edited[:,5] == src) &\
                (collection_array_edited[:,6] == targ))
            other_inds = np.where((collection_array_edited[:,3] == [i[0]]) &\
                (collection_array_edited[:,4] == i[1]) &\
                (collection_array_edited[:,5] == targ) &\
                (collection_array_edited[:,6] == src))

            all_inds = np.append(inds[0], other_inds[0])
            collection_array_edited[all_inds,9] = 'red'

self_transfer_rows = np.where(collection_array_edited[:,5] == collection_array_edited[:,6])
collection_array_edited[self_transfer_rows,9] = 'red'

# add edges to new graph with riskiness flag as attribute
graph = nx.MultiDiGraph()

for row in collection_array_edited:
    src = row[5]
    targ = row[6]
    token = row[4]
    weight = row[7]
    color = row[9]
    graph.add_edge(src, targ, token=token, value=weight, color=color)

# setup visualization with pyvis
from pyvis import network as net
vis = net.Network(height='700px', width='85%', directed=True, filter_menu=True, cdn_resources='remote')
vis.heading = "Graph of transactions for the Pulsquares NFT Collection"
vis.from_nx(graph)

# add titles to nodes
neighbor_map = vis.get_adj_list()
for node in vis.nodes:
    num_risky = 0
    #print(f"\t examining node {node}")
    for i in vis.get_edges():
        #print(f"\t\t examining edge: {i}")
        if i['from'] == node['label'] or i['to'] == node['label']:
            #print(f"\t\t node in examined edge")
            if i['color'] == 'red':
                #print(f"\t\t node {node['label']} sees risky trans!")
                num_risky += 1
    node["title"] = f"{num_risky} associated risky transactions for wallet {node['label']}"
    node['size'] = len(neighbor_map[node['label']])*2
vis.set_edge_smooth('dynamic')
vis.force_atlas_2based()
vis.show('Pulsquares.html')